In [ ]:
def semantic_tokenization(text):
    # Загружаем модель с NER
    doc = nlp_eng(text)
    
    semantic_units = []
    processed_tokens = set()
    
    # 1. Извлекаем именованные сущности
    for ent in doc.ents:
        if ent.label_ in ['PER', 'ORG', 'LOC', 'MONEY', 'DATE']:
            semantic_units.append({
                'text': ent.text,
                'type': 'named_entity',
                'label': ent.label_,
                'start': ent.start,
                'end': ent.end
            })
            processed_tokens.update(range(ent.start, ent.end))
    
    # 2. Извлекаем составные термины (noun phrases)
    for chunk in doc.noun_chunks:
        if len(chunk) > 1 and not any(i in processed_tokens for i in range(chunk.start, chunk.end)):
            semantic_units.append({
                'text': chunk.text,
                'type': 'noun_phrase',
                'start': chunk.start,
                'end': chunk.end
            })
            processed_tokens.update(range(chunk.start, chunk.end))
    
    # 3. Добавляем оставшиеся значимые токены
    for token in doc:
        if (token.i not in processed_tokens and 
            not token.is_punct and 
            not token.is_space and 
            token.pos_ in ['NOUN', 'VERB', 'ADJ', 'NUM']):
            semantic_units.append({
                'text': token.text,
                'type': 'single_token',
                'pos': token.pos_,
                'start': token.i,
                'end': token.i + 1
            })
    
    # Сортируем по позиции в тексте
    semantic_units.sort(key=lambda x: x['start'])
    return semantic_units

In [ ]:
semantic_units = semantic_tokenization(text_eng)
semantic_units

[{'text': '# Methods', 'type': 'noun_phrase', 'start': 0, 'end': 2},
 {'text': 'Refutation',
  'type': 'single_token',
  'pos': 'NOUN',
  'start': 3,
  'end': 4},
 {'text': 'Validation',
  'type': 'single_token',
  'pos': 'NOUN',
  'start': 5,
  'end': 6},
 {'text': 'Aging Mechanisms', 'type': 'noun_phrase', 'start': 7, 'end': 9},
 {'text': 'Markers',
  'type': 'single_token',
  'pos': 'NOUN',
  'start': 10,
  'end': 11},
 {'text': 'the Purposes', 'type': 'noun_phrase', 'start': 12, 'end': 14},
 {'text': 'Biological Immortality',
  'type': 'noun_phrase',
  'start': 15,
  'end': 17},
 {'text': '\n\n## Abstract', 'type': 'noun_phrase', 'start': 18, 'end': 22},
 {'text': 'This article', 'type': 'noun_phrase', 'start': 24, 'end': 26},
 {'text': 'examines',
  'type': 'single_token',
  'pos': 'VERB',
  'start': 27,
  'end': 28},
 {'text': 'modern scientific methods',
  'type': 'noun_phrase',
  'start': 29,
  'end': 32},
 {'text': 'used',
  'type': 'single_token',
  'pos': 'VERB',
  'start': 

In [ ]:
def visualize_semantic_units(text, semantic_units, nlp_model):
    """Создаёт HTML визуализацию семантических единиц"""
    
    # Обрабатываем текст spaCy для получения токенов
    doc = nlp_model(text)
    
    # Цвета для разных типов токенов
    colors = {
        'noun_phrase': '#1a237e',        # Тёмно-синий
        'named_entity': '#e65100',       # Тёмно-оранжевый  
        'single_token': '#4a148c',       # Тёмно-фиолетовый
        'technical_term': '#1b5e20',     # Тёмно-зелёный
        'multiword_expression': '#ff6f00' # Янтарный
    }
    
    border_colors = {
        'noun_phrase': '#3f51b5',        # Индиго
        'named_entity': '#ff9800',       # Оранжевый 
        'single_token': '#9c27b0',       # Фиолетовый
        'technical_term': '#4caf50',     # Зелёный
        'multiword_expression': '#ffc107' # Жёлтый
    }
    
    # Создаём HTML разметку
    html_parts = []
    html_parts.append("""
    <div style="font-family: Arial, sans-serif; line-height: 1.6; margin: 20px;">
        <h3>🔍 Визуализация семантической токенизации</h3>
        <div style="margin-bottom: 20px;">
    """)
    
    # Добавляем легенду
    html_parts.append('<div style="margin-bottom: 15px;"><strong>Легенда:</strong><br>')
    for token_type, color in colors.items():
        border_color = border_colors.get(token_type, '#666')
        html_parts.append(f'''
            <span style="background-color: {color}; border: 2px solid {border_color}; 
                         padding: 2px 6px; margin: 2px; border-radius: 4px; font-size: 12px;">
                {token_type.replace('_', ' ').title()}
            </span>
        ''')
    html_parts.append('</div>')
    
    # Создаём карту позиций токенов -> семантические единицы
    token_to_unit = {}
    for unit in semantic_units:
        for token_pos in range(unit['start'], unit['end']):
            token_to_unit[token_pos] = unit
    
    # Строим размеченный текст
    html_parts.append('<div style="font-size: 16px; line-height: 2;">')
    
    current_unit = None
    for i, token in enumerate(doc):
        # Проверяем, нужно ли открыть новый блок
        if i in token_to_unit:
            unit = token_to_unit[i]
            if unit != current_unit:
                # Закрываем предыдущий блок
                if current_unit is not None:
                    html_parts.append('</span>')
                
                # Открываем новый блок
                token_type = unit['type']
                bg_color = colors.get(token_type, '#f5f5f5')
                border_color = border_colors.get(token_type, '#666')
                
                tooltip = f"Type: {token_type}"
                if 'label' in unit:
                    tooltip += f", Label: {unit['label']}"
                if 'pos' in unit:
                    tooltip += f", POS: {unit['pos']}"
                
                html_parts.append(f'''
                    <span style="background-color: {bg_color}; 
                                 border: 2px solid {border_color}; 
                                 padding: 2px 4px; margin: 1px; 
                                 border-radius: 4px;
                                 cursor: help;" 
                          title="{tooltip}">
                ''')
                current_unit = unit
        elif current_unit is not None:
            # Закрываем блок если токен не входит в единицу
            html_parts.append('</span>')
            current_unit = None
        
        # Добавляем сам токен
        if current_unit is None:
            # Токен без разметки
            html_parts.append(f'<span style="color: #666;">{token.text}</span>')
        else:
            html_parts.append(token.text)
        
        # Добавляем пробел если нужно
        if token.whitespace_:
            html_parts.append(token.whitespace_)
    
    # Закрываем последний блок если нужно
    if current_unit is not None:
        html_parts.append('</span>')
    
    html_parts.append('</div></div></div>')
    
    return ''.join(html_parts)

# 🎯 Применяем к твоим данным
def display_tokenization_results(text, semantic_units, nlp_model):
    """Отображает результаты токенизации"""
    
    # 1. HTML визуализация
    html_viz = visualize_semantic_units(text, semantic_units, nlp_model)
    display(HTML(html_viz))
    
    # 2. Статистика по типам токенов
    stats = {}
    for unit in semantic_units:
        token_type = unit['type']
        if token_type not in stats:
            stats[token_type] = 0
        stats[token_type] += 1
    
    print("\n📊 Статистика токенизации:")
    for token_type, count in sorted(stats.items()):
        print(f"  {token_type.replace('_', ' ').title()}: {count}")
    
    # 3. Примеры каждого типа
    print("\n🔍 Примеры токенов по типам:")
    examples = {}
    for unit in semantic_units:
        token_type = unit['type']
        if token_type not in examples:
            examples[token_type] = []
        if len(examples[token_type]) < 3:  # Показываем до 3 примеров
            examples[token_type].append(unit['text'])
    
    for token_type, example_list in examples.items():
        print(f"  {token_type.replace('_', ' ').title()}: {', '.join(example_list[:3])}")

In [ ]:
display_tokenization_results(text_eng, semantic_units, nlp_eng)


📊 Статистика токенизации:
  Named Entity: 5
  Noun Phrase: 311
  Single Token: 395

🔍 Примеры токенов по типам:
  Noun Phrase: # Methods, Aging Mechanisms, the Purposes
  Single Token: Refutation, Validation, Markers
  Named Entity: 1971, 1961, 1998


## Измерение качества токенизации

In [ ]:
def evaluate_tokenization_quality(text, semantic_units, nlp_model):
    """Автоматическая оценка качества токенизации"""
    
    doc = nlp_model(text)
    total_tokens = len(doc)
    
    # Базовые метрики
    metrics = {
        'coverage': 0,           # Покрытие текста
        'redundancy': 0,         # Избыточность (пересечения)
        'avg_unit_length': 0,    # Средняя длина единицы
        'semantic_density': 0,   # Семантическая плотность
        'type_distribution': {}, # Распределение типов
        'quality_score': 0       # Общий балл качества
    }
    
    # 1. Покрытие текста (% токенов, включённых в семантические единицы)
    covered_tokens = set()
    for unit in semantic_units:
        covered_tokens.update(range(unit['start'], unit['end']))
    
    metrics['coverage'] = len(covered_tokens) / total_tokens * 100
    
    # 2. Проверка избыточности (пересечений)
    overlaps = 0
    for i, unit1 in enumerate(semantic_units):
        for unit2 in semantic_units[i+1:]:
            range1 = set(range(unit1['start'], unit1['end']))
            range2 = set(range(unit2['start'], unit2['end']))
            if range1.intersection(range2):
                overlaps += 1
    
    metrics['redundancy'] = overlaps
    
    # 3. Средняя длина семантических единиц
    unit_lengths = [unit['end'] - unit['start'] for unit in semantic_units]
    metrics['avg_unit_length'] = sum(unit_lengths) / len(unit_lengths) if unit_lengths else 0
    
    # 4. Распределение типов
    for unit in semantic_units:
        unit_type = unit['type']
        metrics['type_distribution'][unit_type] = metrics['type_distribution'].get(unit_type, 0) + 1
    
    # 5. Семантическая плотность (% значимых токенов)
    meaningful_tokens = sum(1 for token in doc if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'PROPN'])
    metrics['semantic_density'] = meaningful_tokens / total_tokens * 100
    
    # 6. Общий балл качества (взвешенная сумма)
    coverage_score = min(metrics['coverage'] / 100, 1.0) * 40  # до 40 баллов
    redundancy_penalty = max(0, 20 - overlaps * 5)           # штраф за пересечения
    length_score = min(metrics['avg_unit_length'] / 3, 10)    # до 10 баллов за длину
    density_score = min(metrics['semantic_density'] / 100, 1.0) * 30  # до 30 баллов
    
    metrics['quality_score'] = coverage_score + redundancy_penalty + length_score + density_score
    
    return metrics

# 📈 Красивый отчёт о качестве
def print_quality_report(metrics):
    """Выводит красивый отчёт о качестве токенизации"""
    
    print("🔍 ОТЧЁТ О КАЧЕСТВЕ ТОКЕНИЗАЦИИ")
    print("=" * 50)
    
    # Общий балл
    score = metrics['quality_score']
    if score >= 80:
        grade = "🟢 Отлично"
    elif score >= 60:
        grade = "🟡 Хорошо"
    elif score >= 40:
        grade = "🟠 Удовлетворительно"
    else:
        grade = "🔴 Плохо"
    
    print(f"📊 Общий балл качества: {score:.1f}/100 {grade}")
    print()
    
    # Детальные метрики
    print("📋 Детальные метрики:")
    print(f"  📏 Покрытие текста: {metrics['coverage']:.1f}%")
    print(f"  🔄 Пересечения единиц: {metrics['redundancy']}")
    print(f"  📐 Средняя длина единицы: {metrics['avg_unit_length']:.1f} токенов")
    print(f"  🎯 Семантическая плотность: {metrics['semantic_density']:.1f}%")
    print()
    
    # Распределение типов
    print("📊 Распределение типов токенов:")
    total_units = sum(metrics['type_distribution'].values())
    for token_type, count in sorted(metrics['type_distribution'].items()):
        percentage = count / total_units * 100
        print(f"  {token_type.replace('_', ' ').title()}: {count} ({percentage:.1f}%)")

In [ ]:
metrics = evaluate_tokenization_quality(text_eng, semantic_units, nlp_eng)
print_quality_report(metrics)

🔍 ОТЧЁТ О КАЧЕСТВЕ ТОКЕНИЗАЦИИ
📊 Общий балл качества: 61.7/100 🟡 Хорошо

📋 Детальные метрики:
  📏 Покрытие текста: 62.4%
  🔄 Пересечения единиц: 0
  📐 Средняя длина единицы: 1.8 токенов
  🎯 Семантическая плотность: 53.8%

📊 Распределение типов токенов:
  Named Entity: 5 (0.7%)
  Noun Phrase: 311 (43.7%)
  Single Token: 395 (55.6%)


## phrase-structure trees

In [ ]:
import nltk
from nltk import CFG
from nltk.parse.generate import generate

grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N
    VP -> V NP
    Det -> 'the'
    N -> 'cat' | 'dog'
    V -> 'chased' | 'saw'
""")
parser = nltk.ChartParser(grammar)
sentence = "the cat chased the dog".split()
for tree in parser.parse(sentence):
    tree.pretty_print()
    tree.draw()

              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
Det      N    V    Det      N 
 |       |    |     |       |  
the     cat chased the     dog



In [7]:
import spacy
import benepar
benepar.download('benepar_en3')
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe("benepar", config={"model": "benepar_en3"})

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\dimka\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [ ]:
# doc = nlp("The cat chased the dog.")
doc = nlp("The time for action is now. It is never too late to do something.")
print(list(doc.sents)[0]._.parse_string)
print(list(doc.sents)[1]._.parse_string)

(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
